In [225]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
import matplotlib.patches as mpatches
import rasterio
from rasterio.transform import xy

pd.set_option('display.max_columns', None)

In [226]:
"""
    These data have some errors, as a result, the output files 'analysis/data/level2-adm2-risk_with_context.csv' needs 
    some manual fixing. So DO NOT use this, this is just for the purpose of displaying the code, please use 
    'analysis/data/level2-adm2-risk_with_context.csv' instead, they are the same as these 'levelX-admX-risk.csv' files
    but with more information
"""

# level1 = pd.read_csv('analysis/data/level1-adm1-risk.csv')
# level2 = pd.read_csv('analysis/data/level2-adm2-risk.csv')
# level3 = pd.read_csv('analysis/data/level3-grid-risk.csv')
level3_terrain = pd.read_csv('analysis/data/terrain/grid_terrain_proportion.csv')
level1_population = pd.read_csv('analysis/data/population/adm1_population.csv')

In [227]:
level1.head(3)

,ADM1_id,estimated_uxo,weighted_risk_score,mission_density,risk_level,is_hotspot,geometry,shapeName
0,63759600B11108179203280,1.836,0.000082,2.909091,Low,False,POLYGON ((106.19241959119125 22.38437988162627...,Cao Bằng
1,63759600B12588044805894,24782.171,0.016468,195.671233,Very High,True,MULTIPOLYGON (((106.80830528925776 10.39326411...,Tiền Giang
2,63759600B12839449494422,34197.142,0.008533,97.245847,Very High,True,POLYGON ((107.4922969120336 10.719643141289287...,Đồng Nai


In [228]:
level2.head(3)

,ADM2_id,grid_count,estimated_uxo,weighted_risk_score,average_risk_score,mission_density,weapons_density,weighted_avg_risk_level,ADM1_name,ADM1_id,ADM2_name,geometry,adm2_risk
0,81297802B10167820950608,50,78.605,89,0.000376,277,2697,1.780000,Bắc Giang,63759600B37262939240335,Son Dong,POLYGON ((107.03241001800006 21.32525058600004...,Moderate
1,81297802B1021983329449,24,51.105,43,0.000359,106,1221,1.791667,Cần Thơ,63759600B21046589863912,Co Do,POLYGON ((105.55657725400005 10.15214766600007...,Moderate
2,81297802B10372149063277,56,1248.560,211,0.003480,2319,22260,3.767857,Bình Thuận,63759600B31473825670333,Duc Linh,POLYGON ((107.54319090500007 11.36206983900007...,High


In [229]:
level3.head(3)

,grid_id,est_dud_weapon,mission_count,weapons_delivered,ADM1_name,ADM1_id,ADM2_name,ADM2_id,grid_center_lon,grid_center_lat,risk_score,risk_level
0,100_729,0.00,1,0,Sơn La,63759600B3920147428024,Sop Cop,81297802B35904575630157,103.248875,20.853103,0.000000,Low
1,100_731,0.00,2,0,Điện Biên,63759600B72990231521734,Dien Bien,81297802B1284717956892,103.248213,20.910223,0.000030,Low
2,100_732,28.48,13,338,Điện Biên,63759600B72990231521734,Dien Bien,81297802B1284717956892,103.247881,20.938783,0.002733,High


In [230]:
level3_terrain.head(3)

,grid_id,residence developed,residence,agricultural,open_terrain,shrubland,forest,wetlands_water
0,100_729,0.0,0.0,0.013699,0.169652,0.0,0.816649,0.0
1,100_731,0.0,0.0,0.034277,0.190760,0.0,0.774963,0.0
2,100_732,0.0,0.0,0.022398,0.108037,0.0,0.869565,0.0


In [231]:
level1_population.head(3)

,city,population
0,Hà Nội,"8,587,081"
1,Hà Giang,"899,900"
2,Cao Bằng,"547,857"


# Integrating contextual data 

### Level 3
- At level 3, we can directly integrate environmental data

In [232]:
level3 = level3.merge(level3_terrain, how='left', on='grid_id')

In [233]:
# level3.to_csv('analysis/data/level3-grid-risk_with_context.csv', index=False)
level3.head(5)

,grid_id,est_dud_weapon,mission_count,weapons_delivered,ADM1_name,ADM1_id,ADM2_name,ADM2_id,grid_center_lon,grid_center_lat,risk_score,risk_level,residence developed,residence,agricultural,open_terrain,shrubland,forest,wetlands_water
0,100_729,0.00,1,0,Sơn La,63759600B3920147428024,Sop Cop,81297802B35904575630157,103.248875,20.853103,0.000000,Low,0.0,0.0,0.013699,0.169652,0.0,0.816649,0.0
1,100_731,0.00,2,0,Điện Biên,63759600B72990231521734,Dien Bien,81297802B1284717956892,103.248213,20.910223,0.000030,Low,0.0,0.0,0.034277,0.190760,0.0,0.774963,0.0
2,100_732,28.48,13,338,Điện Biên,63759600B72990231521734,Dien Bien,81297802B1284717956892,103.247881,20.938783,0.002733,High,0.0,0.0,0.022398,0.108037,0.0,0.869565,0.0
3,100_737,0.00,3,0,Điện Biên,63759600B72990231521734,Dien Bien Dong,81297802B35707987060986,103.246212,21.081582,0.000060,Low,0.0,0.0,0.012209,0.329634,0.0,0.658158,0.0
4,100_740,0.60,3,6,Điện Biên,63759600B72990231521734,Dien Bien Dong,81297802B35707987060986,103.245204,21.167260,0.000107,Low,0.0,0.0,0.031930,0.400581,0.0,0.567489,0.0


### Level 2

In [234]:
import pandas as pd

# Constants
GRID_AREA_KM2 = 10

# Terrain proportion columns
terrain_cols = [
    'residence developed',
    'residence',
    'agricultural',
    'open_terrain',
    'shrubland',
    'forest',
    'wetlands_water'
]

for col in terrain_cols:
    level3[f'area_{col}'] = level3[col] * GRID_AREA_KM2

area_cols = [f'area_{col}' for col in terrain_cols]
level3_agg = level3.groupby('ADM2_id')[area_cols].sum().reset_index()

level2 = level2.merge(level3_agg, on='ADM2_id', how='left')

level2['area(km2)'] = level2['grid_count'] * GRID_AREA_KM2

for col in terrain_cols:
    level2[col] = level2[f'area_{col}'] / level2['area(km2)']

level2.drop(columns=area_cols, inplace=True)


In [ ]:
# level2.to_csv('analysis/data/level2-adm2-risk_with_context.csv', index=False)
# level2.head(1)

### Level 1

In [236]:
level1 = level1.rename(columns={'shapeName' : 'city'})
level1 = level1.merge(level1_population, 'left', 'city')

In [239]:
level1.head(2)

,ADM1_id,estimated_uxo,weighted_risk_score,mission_density,risk_level,is_hotspot,geometry,city,population
0,63759600B11108179203280,1.836,0.000082,2.909091,Low,False,POLYGON ((106.19241959119125 22.38437988162627...,Cao Bằng,"547,857"
1,63759600B12588044805894,24782.171,0.016468,195.671233,Very High,True,MULTIPOLYGON (((106.80830528925776 10.39326411...,Tiền Giang,"1,790,653"


In [238]:
level1[level1['population'].isnull()]

,ADM1_id,estimated_uxo,weighted_risk_score,mission_density,risk_level,is_hotspot,geometry,city,population


In [240]:
terrain_cols = [
    'residence developed',
    'residence',
    'agricultural',
    'open_terrain',
    'shrubland',
    'forest',
    'wetlands_water'
]

for col in terrain_cols:
    level2[f'area_{col}'] = level2[col] * level2['area(km2)']

level1_agg = level2.groupby('ADM1_id')[
    [f'area_{col}' for col in terrain_cols] + ['area(km2)']
].sum().reset_index()

for col in terrain_cols:
    level1_agg[col] = level1_agg[f'area_{col}'] / level1_agg['area(km2)']

level1_agg.drop(columns=[f'area_{col}' for col in terrain_cols], inplace=True)

level1 = level1.merge(level1_agg, on='ADM1_id', how='left')

In [242]:
# level1.to_csv('analysis/data/level1-adm1-risk_with_context.csv', index=False)